In [1]:
import os
import pandas as pd

### Iteration 1

In [8]:
folder_path = "/Users/jenniferzhang/Desktop/Risk Lab Research/epi2024_rawdata06022024"
# get the list of files in the folder
file_names = os.listdir(folder_path)

# initialize
na_file_names = [] # only get the na files
starting_years = []
ending_years = []

for file in file_names:
    if "na" in file:
        # file name
        na_file_names.append(file)
        # read csv file
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path) 
        header = df.columns # get the column names
        # starting year
        starting_year = str(header[3])[-4:]
        # ending year
        ending_year = str(header[-1])[-4:]
        starting_years.append(starting_year)
        ending_years.append(ending_year)

# resulting dataframe
result_df = pd.DataFrame({
    "File Names": na_file_names,
    "Starting Year": starting_years,
    "Ending Year": ending_years
})
print(result_df)

         File Names Starting Year Ending Year
0    PDN_raw_na.csv          1960        2022
1        VOE_na.csv          2003        2022
2    CBP_raw_na.csv          2022        2022
3    HPE_raw_na.csv          1998        2022
4        PME_na.csv          1998        2022
..              ...           ...         ...
118  PF5_raw_na.csv          2006        2022
119  PAE_raw_na.csv          2022        2022
120  PRS_raw_na.csv          2015        2018
121  HFX_raw_na.csv          1990        2020
122  LUE_raw_na.csv          1990        2022

[123 rows x 3 columns]


In [23]:
# get only the indicator abbreviations from the file names
names = list(result_df["File Names"])
indicators = []
for n in names:
    indicators.append(n[:3])
ind = set(indicators) # unique indicators

# in the data_catelogue excel file
# add abbreviation
abbre = []
for i in result_df["File Names"]:
        abbre.append(i[:3])
result_df["Abbreviation"] = abbre

# add description if exists
variabe_df = pd.readcsv("/Users/jenniferzhang/Desktop/Risk Lab Research/epi20224ariables.csv")
var_des = variabe_df[["Abbreviation","Variable"]]
var_des_dict = var_des.set_index("Abbreviation")["Variable"].to_dict()
# define function
def mapping(a):
    return var_des_dict.get(a, "Unknown")
# apply function to "Abbreviation" column
result_df["Description"] = result_df["Abbreviation"].apply(mapping)
result_df
        

,File Names,Starting Year,Ending Year,Abbreviation,Description
0,PDN_raw_na.csv,1960,2022,PDN,Unknown
1,VOE_na.csv,2003,2022,VOE,VOC exposure
2,CBP_raw_na.csv,2022,2022,CBP,Projected cumulative emissions to 2050 relativ...
3,HPE_raw_na.csv,1998,2022,HPE,Anthropogenic PM2.5 exposure
4,PME_na.csv,1998,2022,PME,Unknown
...,...,...,...,...,...
118,PF5_raw_na.csv,2006,2022,PF5,Unknown
119,PAE_raw_na.csv,2022,2022,PAE,Protected Area Effectiveness
120,PRS_raw_na.csv,2015,2018,PRS,Pesticide Pollution Risk
121,HFX_raw_na.csv,1990,2020,HFX,Unknown


In [25]:
# input the data into excel
excel_path = "/Users/jenniferzhang/Desktop/Risk Lab Research/data_catalogue.xlsx"
result_df.to_excel(excel_path, index=False)

### Iteration 2

In [2]:
ff = pd.read_excel("/Users/jenniferzhang/Desktop/Risk Lab Research/data_catalogue.xlsx")
ff.head()

,File Names,Starting Year,Ending Year,Abbreviation,Description
0,PDN_raw_na.csv,1960,2022,PDN,Unknown
1,VOE_na.csv,2003,2022,VOE,VOC exposure
2,CBP_raw_na.csv,2022,2022,CBP,Projected cumulative emissions to 2050 relativ...
3,HPE_raw_na.csv,1998,2022,HPE,Anthropogenic PM2.5 exposure
4,PME_na.csv,1998,2022,PME,Unknown


In [7]:
epi_2022 = pd.read_excel("/Users/jenniferzhang/Desktop/Risk Lab Research/epi2022variables.xlsx", header=1)
var_df = epi_2022[["Variable", "Abbreviation"]]
var_df.head()

,Variable,Abbreviation
0,GDP,GDP
1,GDP per capita,GPC
2,Population,POP
3,GDP growth rate,GDB
4,Population Density,PDN


In [17]:
for i, item in var_df.iterrows():
    if item["Abbreviation"] in ff["Abbreviation"].values:
        ff.loc[ff['Abbreviation'] == item["Abbreviation"], 'Description'] = item["Variable"]

In [19]:
ff.head()

,File Names,Starting Year,Ending Year,Abbreviation,Description
0,PDN_raw_na.csv,1960,2022,PDN,Population Density
1,VOE_na.csv,2003,2022,VOE,Volatile Organic Compound Exposure
2,CBP_raw_na.csv,2022,2022,CBP,Projected cumulative emissions to 2050 relativ...
3,HPE_raw_na.csv,1998,2022,HPE,Anthropogenic PM2.5 exposure
4,PME_na.csv,1998,2022,PME,Unknown


In [20]:
# input the data into excel
excel_path = "/Users/jenniferzhang/Desktop/Risk Lab Research/data_catalogue.xlsx"
ff.to_excel(excel_path, index=False)

In [24]:
### Copy the files with known description into a seperate file
import os
import shutil

source_dir = '/Users/jenniferzhang/Desktop/Risk Lab Research/epi2024_rawdata06022024'  
destination_dir = '/Users/jenniferzhang/Desktop/Risk Lab Research/epi2024_processeddata' 

known_file = []
for i, item in ff.iterrows():
    if item["Description"] != "Unknown":
        known_file.append(item["File Names"])

for filename in known_file:
    source_file = os.path.join(source_dir, filename)
    if os.path.exists(source_file):
        destination_file = os.path.join(destination_dir, filename)
        shutil.copy(source_file, destination_file)
        print(f"File '{filename}' copied successfully.")
    else:
        print(f"File '{filename}' not found in source directory.")

File 'PDN_raw_na.csv' copied successfully.
File 'VOE_na.csv' copied successfully.
File 'CBP_raw_na.csv' copied successfully.
File 'HPE_raw_na.csv' copied successfully.
File 'LED_raw_na.csv' copied successfully.
File 'TC5_raw_na.csv' copied successfully.
File 'LUF_raw_na.csv' copied successfully.
File 'RMS_raw_na.csv' copied successfully.
File 'NXA_raw_na.csv' copied successfully.
File 'FCL_raw_na.csv' copied successfully.
File 'CDA_raw_na.csv' copied successfully.
File 'OEB_raw_na.csv' copied successfully.
File 'NOE_raw_na.csv' copied successfully.
File 'TCC_raw_na.csv' copied successfully.
File 'GHG_raw_na.csv' copied successfully.
File 'COE_na.csv' copied successfully.
File 'USD_raw_na.csv' copied successfully.
File 'RLI_raw_na.csv' copied successfully.
File 'VOE_raw_na.csv' copied successfully.
File 'GDB_raw_na.csv' copied successfully.
File 'WWC_raw_na.csv' copied successfully.
File 'FCD_raw_na.csv' copied successfully.
File 'CHA_raw_na.csv' copied successfully.
File 'GPC_raw_na.cs